In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: alexisjihyeross. Use `wandb login --relogin` to force relogin


True

In [5]:
import pandas as pd 
import wandb
import numpy as np

api = wandb.Api(timeout=50)
entity, project = "lm-informants", "maybefinal_four"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [6]:
df = pd.DataFrame()

In [7]:
from tqdm import tqdm

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run in tqdm(runs): 
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        ]
                
    df = pd.DataFrame()
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run: ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})

    num_steps = len(exp[keys[0]])
    for k in keys:
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        
    all_data.extend(results)
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


  0%|                                                                                                                  | 1/2177 [00:00<07:53,  4.60it/s]

filtering run:  ['lm-informants', 'maybefinal_four', 'uyqzqhi2']


  3%|███▏                                                                                                             | 62/2177 [00:26<13:22,  2.63it/s]

filtering run:  ['lm-informants', 'maybefinal_four', '806t0tmn']


 18%|████████████████████▍                                                                                           | 398/2177 [02:56<13:46,  2.15it/s]

filtering run:  ['lm-informants', 'maybefinal_four', '21gmklhf']


 42%|███████████████████████████████████████████████▌                                                                | 925/2177 [07:33<09:39,  2.16it/s]

filtering run:  ['lm-informants', 'maybefinal_four', 'oiuh34ad']


2179it [18:20,  1.98it/s]                                                                                                                               


In [9]:
display(mega_df)

,step,auc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/log_log_alpha_ratio,strategy,wandb_id
0,0.0,0.752471,1,0,kl_train_mixed,0.001,0.05,False,atr_harmony,None,symmetric,0.1,kl_train_mixed,"[lm-informants, maybefinal_four, psswi0lj]"
1,1.0,0.751773,1,0,kl_train_mixed,0.001,0.05,False,atr_harmony,None,symmetric,0.1,kl_train_mixed,"[lm-informants, maybefinal_four, psswi0lj]"
2,2.0,0.708094,1,0,kl_train_mixed,0.001,0.05,False,atr_harmony,None,symmetric,0.1,kl_train_mixed,"[lm-informants, maybefinal_four, psswi0lj]"
3,3.0,0.721192,1,0,kl_train_mixed,0.001,0.05,False,atr_harmony,None,symmetric,0.1,kl_train_mixed,"[lm-informants, maybefinal_four, psswi0lj]"
4,4.0,0.719097,1,0,kl_train_mixed,0.001,0.05,False,atr_harmony,None,symmetric,0.1,kl_train_mixed,"[lm-informants, maybefinal_four, psswi0lj]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326245,145.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
326246,146.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
326247,147.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
326248,148.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"


In [10]:
out_path = './runs.csv'
mega_df.to_csv(out_path)